# The EZ-RPG Support System

The
EZ-RPG
aims to be a
*RPG-system neutral*
helper for
both
*Game Masters*
and
*Players*.

EZ-RPG mostly focuses on
*keeping immutable data about characters*
(think
"strength"
not
"hit points")
and rolling dice baed on the data.
It is especially useful when using a friendly Python console.

There are many consoles:
`ptpython`
and
`bpython`,
among others.
One of the most popular ones is Jupyter Notebook,
which will be used in these examples.


## Setup

The first step is to import the relevant modules:

In [1]:
import logging
import functools
import random

In [2]:
from ezrpg import character, sheet

After that,
it is highly useful to set two global things:

* A function that will create characters from
  TOML
  character sheets.
* Logging at the
  `INFO`
  level.

In [3]:
from_toml = functools.partial(
    sheet.from_toml,
    character.dice_maker(random.Random()),
)
logging.basicConfig(level=logging.INFO)

## Setting up characters

Often,
your character data will be in a TOML
in a separate file
(or maybe even a database,
for sophisticated use-cases!)
For ease of explanation,
here,
it will be embedded online.

As the example,
we use a character built using the
[Fate](https://www.faterpg.com/)
system.
This helps make sure this docmentation
does not run afoul of licensing issues.

The character has a
"Lore"
*skill*
and an
"Arcane Lore"
*stunt*.
The stunt gives a bonus to
Lore
skill use which pertain to the Arcane.

In [7]:
my_character = from_toml("""
[general]
name = "Example Character"
Arcane_Lore = 2

[moves]
lore = {}
lore_arcane = {}

[moves.default]
roll="4d3"
succeed=">-1"
effect=1

[[moves.default.adjustments]]
constant = -8

[[moves.default.bonus_effect]]
level = 1
value = 2

[[moves.default.bonus_effect]]
level = 3
value = 3

[[moves.lore_arcane.adjustments]]
trait = "Arcane_Lore"
""")

There is a lot to unpack here!
First, the part that helps specify the
"Fate"
skill system:
you roll 4 3-sided dice and add them.

However,
the dice have
`+1`/`-1`/`-0`
faces
instead of
`1/2/3`
faces.
Note that subtracting
`2`
from a die gives equivalent result.
Subtracting
`2 * 4 = 8`
from the sum achieves the original goal.

In the Fate system,
rolling a negative number is failing
(the effect size will be "0").
Just making it means a "tie"
(effect size of 1),
making it by 1 or 2 is
"success"
and 3 or more is
"spectacular success".

This helps explain

```toml
[moves.default]
roll="4d3"
succeed=">-1"
effect=1

[[moves.default.adjustments]]
constant = -8

[[moves.default.bonus_effect]]
level = 1
value = 2

[[moves.default.bonus_effect]]
level = 3
value = 3
```

This shows how to use different
RPGs
with the
EZ-RPG games.
Note that in general,
effect sizes can also be random
(e.g.,
`3d6+5`).

The rest of the sheet is the specific character itself:

```toml
[general]
name = "Example Character"
Arcane_Lore = 2

[moves]
lore = {}
lore_arcane = {}

[[moves.lore_arcane.adjustments]]
trait = "Arcane_Lore"
```

The
`Arcane_Lore`
represents the stunt,
while the two
"moves"
represent using the
Lore
skill
and using it
for Arcane purposes.
The Arcane one gets an
*adjustment*
based on the
`Arcane_Lore`
stunt.

## Looking at characters

The default presentation of characters shows traits and moves,
without the rules.
This is designed for
*role*
playing,
not
*roll*
playing:
checking up on the character:

In [12]:
my_character

Arcane_Lore,2
lore,
lore_arcane,


## Using moves

Using the
"moves"
(skills, attacks, even passive defenses)
is the dice-rolling assistance.
As a helper,
encode the Fate
dice-rolling rules:

In [11]:
FATE_LADDER = {
    0: "Fail",
    1: "Tie",
    2: "Succeed",
    3: "Succeed spectacularly",
}

Now,
the character can use the
Lore
skill on a mediocre target:

In [13]:
FATE_LADDER[int(my_character.moves.lore)]

INFO:ezrpg.character:Adjusted threshold: Threshold(threshold_dice=_Dice(num=4, value=3, _random=<random.Random object at 0xaaaaed7ca230>, constant=0), effect=True, no_effect=0, bonus_effect={1: 2, 3: 3}, maximum=None, minimum=7)
INFO:ezrpg.character:Rolled _Dice(num=4, value=3, _random=<random.Random object at 0xaaaaed7ca230>, constant=0), got 8
INFO:ezrpg.character:Succeeded by 1


'Succeed'

Note that the result is the
"result",
but the details about rolling and numbers are logged.
This is why enabling
`INFO`
logging
is useful:
these details can be useful for crits and edge-cases.